In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import os


In [0]:
pip install Distance

     |████████████████████████████████| 184kB 2.6MB/s 
  Created wheel for Distance: filename=Distance-0.1.3-cp36-none-any.whl size=16261 sha256=3939f6c95930aaa2a48fac3eabded4478afd78b67e9b1346f083bfeb7dc809ce
  Stored in directory: /root/.cache/pip/wheels/d5/aa/e1/dbba9e7b6d397d645d0f12db1c66dbae9c5442b39b001db18e
Successfully built Distance


In [0]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import re
import time
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from scipy.sparse import csr_matrix
import re
import string
import time
import json
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame
from collections import OrderedDict
from scipy.sparse import csr_matrix
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import distance
from sklearn.linear_model import SGDClassifier

In [0]:
import pickle

In [0]:
train_df = pickle.load(open(r"/content/drive/My Drive/Colab Notebooks/Statistical Computing/train_df.pkl", "rb"))
test_df = pickle.load(open(r"/content/drive/My Drive/Colab Notebooks/Statistical Computing/test_df.pkl", "rb"))

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(binary=True).fit(train_df['seperated_ingredients'].values)
X_train_vectorized = vect.transform(train_df['seperated_ingredients'].values)
X_train_vectorized = X_train_vectorized.astype('float')
Result_transformed = vect.transform(test_df['seperated_ingredients'].values)
Result_transformed = Result_transformed.astype('float')

In [0]:
vect

TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
print(X_train_vectorized.shape)

(39774, 2811)


In [0]:
print(Result_transformed.shape)

(9944, 2811)


In [0]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_transformed = encoder.fit_transform(train_df.cuisine)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_vectorized, y_transformed , random_state = 0)

Grid Search

In [0]:
params_grid_rf = {'bootstrap' : ['True'], 'max_depth':[10, 12, 15, 18, None],'max_features':['auto','sqrt'],
               'min_samples_split':[4,3,5], 'n_estimators':[500,800, 1200], 'criterion':['gini']}
rf = RandomForestClassifier(n_jobs=-1)

gsearch_rf = GridSearchCV(estimator = rf, param_grid=params_grid_rf, cv=5, verbose=3)

rf_grid = gsearch_rf.fit(X_train , y_train)

[CV]  bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_split=5, n_estimators=1200, score=0.748, total= 4.4min
[CV] bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_split=5, n_estimators=1200 
[CV]  bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_split=5, n_estimators=1200, score=0.736, total= 4.4min
[CV] bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_split=5, n_estimators=1200 
[CV]  bootstrap=True, criterion=gini, max_depth=None, max_features=auto, min_samples_split=5, n_estimators=1200, score=0.742, total= 4.4min
[CV] bootstrap=True, criterion=gini, max_depth=None, max_features=sqrt, min_samples_split=4, n_estimators=500 
[CV]  bootstrap=True, criterion=gini, max_depth=None, max_features=sqrt, min_samples_split=4, n_estimators=500, score=0.747, total= 1.9min
[CV] bootstrap=True, criterion=gini, max_depth=None, max_features=sqrt, min_samples_split=4, n_estimators=

[Parallel(n_jobs=1)]: Done 450 out of 450 | elapsed: 479.3min finished


In [0]:
rf_grid.best_params_

{'bootstrap': 'True',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_split': 3,
 'n_estimators': 800}

In [0]:
best_params = {'bootstrap': 'True',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_split': 3,
 'n_estimators': 800}

rf_best_params = RandomForestClassifier(**best_params)

rf_fit = rf_best_params.fit(X_train_vectorized, y_transformed)

In [0]:
y_predicted = rf_best_params.predict(Result_transformed)
y_predicted_final = encoder.inverse_transform(y_predicted)
predictions = pd.DataFrame({'cuisine' : y_predicted_final , 'id' : test_df.id })
predictions = predictions[[ 'id' , 'cuisine']]
predictions.to_csv(r'/content/drive/My Drive/Colab Notebooks/Statistical Computing/submit_RF_best_params.csv', index = False) # 0.74859